# Поиск наиболее близкого имени участника

In [1]:
# conda install -c conda-forge python-levenshtein
# или pip install python-levenshtein

import Levenshtein

In [2]:
# что было написано в заявке

claim_list = [
    'Шехавцова Анна',
    'Гречихина Наталья',
    'Козлова Алена',
    'Груздева Алина',
    'Кущенко Анна',
    'Чистякова Анастасия' 
]

In [3]:
# результат расшифровки речи диктора

speech_recognition = [
    'кучменко она',
    'кущенко оксана',
    'груздь алина',
    'рычихина наталья',
    'шиховцева на',
    'чистова анастасия'
]

Считаем расстояние левенштейна для всех комбинаций, составляем и сортируем словарь. В итоговый список добавляем только первые элементы словаря (с наименьшим расстоянием, то есть с наименьшей вероятностью ошибки распознавания речи).

In [4]:
list_ = []
for recognized_name in speech_recognition:
    distances = {real_name: Levenshtein.distance(recognized_name, real_name) for real_name in claim_list}
    distances = sorted(distances.items(), key = lambda x: x[1])
    list_.append(distances[0][0])
list_
    
    

['Кущенко Анна',
 'Кущенко Анна',
 'Груздева Алина',
 'Гречихина Наталья',
 'Шехавцова Анна',
 'Чистякова Анастасия']

Ок, получили список, можно остановиться. Но в списке повторы - кого-то диктор не упомянул. Нехорошо. Попытаемся добавлять в список неповторяющиеся элементы. То есть, если элемент в списке уже есть, добавим следующий по расстоянию левенштейна элемент.

In [5]:
list_1 = []
for recognized_name in speech_recognition:
    distances = {real_name: Levenshtein.distance(recognized_name, real_name) for real_name in claim_list}
    distances = sorted(distances.items(), key = lambda x: x[1])
    for num in range(len(distances)):
        if not distances[num][0] in list_1:
            list_1.append(distances[num][0])
            break
        else:
            num += 1
list_1
    
    

['Кущенко Анна',
 'Шехавцова Анна',
 'Груздева Алина',
 'Гречихина Наталья',
 'Козлова Алена',
 'Чистякова Анастасия']

Ок, повторов нет, но после первого элемента вышло много ошибок (вернее, все определены ошибочно). Нужно действовать иначе.

Решим проблему в общем случае:
1. Создаем счетчик повторений имен.
2. Если повторений больше 1 (т.е. в принципе есть повторения), составим список повторений.
3. Удалим все элементы, которые распознал наш несовершенный "речевой анализатор" из изначального списка спортсменов. Если все верно, то должно остаться ровно столько неупомянутых спортсменов, сколько было повторений.
4. Заполним все повторные вхождения оставшимися спортсменами.
5. Теперь мы имеем список без повторений. Однако сами повторения в нем вставлены рандомно. Можно оптимизировать, считая попарно расстояния левенштейна для повторов или что то еще. Но пока пусть работает так - проблема на стороне распознавателя речи :)

In [6]:
from collections import Counter
counter = Counter(list_)

In [7]:
d = sorted(dict(counter).items(), key = lambda x: -x[1])
duplicates = []
init_list = claim_list
for i in d:
    init_list.remove(i[0])
for i in d:
    if not i[1] == 1:
        num_duplicates = i[1]
        duplicate = i[0]
        duplicates.append([duplicate,num_duplicates])
duplicates

[['Кущенко Анна', 2]]

In [8]:
for i in range(len(duplicates)):
    for j in range(1,duplicates[i][1]):
        num = [a for a, n in enumerate(list_) if n == duplicates[i][0]][j]
        list_[num] = init_list[0]
        init_list.remove(init_list[0])
list_

['Кущенко Анна',
 'Козлова Алена',
 'Груздева Алина',
 'Гречихина Наталья',
 'Шехавцова Анна',
 'Чистякова Анастасия']